# Inference

In [1]:
# Ensure local package is importable in this notebook (editable install) and add project root to sys.path.
from pathlib import Path
import sys

project_root = str(Path("..").resolve())
if project_root not in sys.path:
	sys.path.insert(0, project_root)

from src.models.ebm_predictor import EBMPredictor

In [2]:
# FPKM coverage files path
base_path = '/mnt/d/flync_final_runs/Vu_GSE199164_3d_vs_7d/vu_gse199164/cov/'

In [3]:
predictor = EBMPredictor(
    model_path="/home/chlab/flync/final_train_artifacts/FINAL_flync_ebm_model.pkl",
    schema_path="/home/chlab/flync/final_train_artifacts/flync_model_schema.json",
    check_value_ranges=True
)

In [4]:
import pandas as pd

inference = pd.read_parquet("cleaned_dataset_3d_7d.parquet")
res = pd.read_parquet('features.parquet').set_index('index')

inference.shape, res.shape

((24219, 731), (27021, 43))

In [5]:
res_cols = ['transcript_id', 'chromosome', 'start', 'end', 'ss_sequence', 'ss_structure']

res = res[[c for c in res.columns if c in res_cols]]

In [6]:
res

,chromosome,start,end,ss_sequence,ss_structure
index,,,,,
MSTRG.2.1,chr2L,9837,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,.((((((.((((((((((((.....))))))).(((((((((((.....
MSTRG.2.2,chr2L,9838,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,.((((((.((((((((((((.....))))))).(((((((((((.....
MSTRG.2.14,chr2L,9842,20015,AAUAAAUUCGGCCACAAUUAAUAUACUUAUUUAUUCAUCAAAACCA...,.......((((((...........(((..((((((((((((((......
MSTRG.2.16,chr2L,10953,21424,AAAUAUACCUUCUAAAAAAUAUAUGAUUUUUUCGAUAGAGAUACAC...,...................((((((........(((((((((((.....
MSTRG.2.17,chr2L,13484,20015,AAUAAAUUCGGCCACAAUUAAUAUACUUAUUUAUUCAUCAAAACCA...,.........((((..............((((((((((((((((......
...,...,...,...,...,...
MSTRG.24539.1,chrY,3637723,3640688,GUGGAAAAAUAUUCUUAGGAAUUACCAAACACUAUAAAAACAAAAU...,(((.......((((....)))).......)))...((((((........
MSTRG.24540.1,chrY,3648684,3649117,CGUCUAAUCAGUAUAUGUGUACAAAAAGAACGAAUAAAGAACGGUA...,.......((((.((((.(((.((((...............((((((...
MSTRG.24541.2,chrY,3665131,3666483,GAAGUGGAGCAGGUGUAGAAAUAGGAAGAGAACUAGGGGUAAAUAG...,...(((((((.((((((((..(((((((.(((...(((((...(((...


In [7]:
inference["proba"] = pd.DataFrame(predictor.predict_proba(inference, validate=False)[:, 1], index=inference.index)

In [8]:
inference["pred"] = predictor.predict(inference.drop(columns=["proba"]), validate=False)

In [9]:
res = res.merge(inference[['proba', 'pred']], how='right', left_index=True, right_index=True)

In [10]:
import pandas as pd
from pathlib import Path

sources = Path(base_path).glob('*/t_data.ctab')

paths = {p.parent.name: p for p in sources}

def parse_ctabs(paths: dict[str, Path]) -> pd.DataFrame:
    dfs = []
    for name, path in paths.items():
        df = pd.read_csv(path, sep='\t', usecols=['t_name', 'FPKM'])
        df = df.rename(columns={'FPKM': f'fpkm_{name.lower()}'})
        df.drop_duplicates(subset=['t_name'], inplace=True)
        dfs.append(df)
    merged_df = dfs[0]
    for df in dfs[1:]:
        merged_df = merged_df.merge(df, on='t_name', how='outer')
    return merged_df

def fpkm_stats(fpkm_df: pd.DataFrame) -> pd.DataFrame:
    fpkm_values = fpkm_df.drop(columns=['t_name'])
    fpkm_df['fpkm_mean'] = fpkm_values.mean(axis=1)
    fpkm_df['fpkm_median'] = fpkm_values.median(axis=1)
    fpkm_df['fpkm_std'] = fpkm_values.std(axis=1)
    return fpkm_df

fpkm = fpkm_stats(parse_ctabs(paths))

In [12]:
fpkm.set_index('t_name', inplace=True)

In [13]:
fpkm

,fpkm_srr18432800,fpkm_srr18432801,fpkm_srr18432802,fpkm_srr18432803,fpkm_srr18432804,fpkm_srr18432805,fpkm_srr18432809,fpkm_srr18432810,fpkm_srr18432811,fpkm_srr18432812,fpkm_srr18432813,fpkm_srr18432814,fpkm_mean,fpkm_median,fpkm_std
t_name,,,,,,,,,,,,,,,
FBgn0013687_df_pt,0.059986,0.047586,0.000000,0.023756,0.038929,0.085387,0.000000,0.033986,0.000000,0.055654,0.000000,0.026073,0.030946,0.030030,0.028080
FBti0015567-RA,1.683359,1.241024,1.494745,1.683366,1.624841,2.337746,0.000000,1.774932,0.920289,1.211146,0.507317,0.815212,1.274498,1.367884,0.635393
FBti0018861-RA,4.445781,2.787519,2.209614,3.244210,2.841586,4.435802,3.991360,2.854652,2.556868,2.029114,1.563118,1.898288,2.904826,2.814553,0.963847
FBti0018862-RA,5.336230,3.416794,2.514294,3.818548,3.420164,5.232962,2.558433,3.720513,2.927633,2.447002,1.892881,2.430595,3.309671,3.172213,1.092313
FBti0018865-RA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RR50960_transposable_element-RA,3.668359,2.762678,1.644487,2.764889,1.990396,3.413096,0.000000,3.526203,3.271206,1.780352,2.373802,2.413094,2.467380,2.587886,1.033794
RR51001_transposable_element-RA,0.059425,0.766703,0.102183,0.366552,0.234558,0.033299,0.000000,0.432333,0.231182,0.138013,0.314431,0.031787,0.225872,0.184597,0.221432
RR51007_transposable_element-RA,7.043255,7.349313,7.286599,6.983126,5.414223,6.882892,2.204550,6.508019,14.137106,7.305468,8.886226,9.805453,7.483853,7.164927,2.789978


In [14]:
res = res.merge(fpkm, how='left', left_index=True, right_index=True)

In [16]:
res

,chromosome,start,end,ss_sequence,ss_structure,proba,pred,fpkm_srr18432800,fpkm_srr18432801,fpkm_srr18432802,...,fpkm_srr18432805,fpkm_srr18432809,fpkm_srr18432810,fpkm_srr18432811,fpkm_srr18432812,fpkm_srr18432813,fpkm_srr18432814,fpkm_mean,fpkm_median,fpkm_std
index,,,,,,,,,,,,,,,,,,,,,
MSTRG.1000.1,chr2L,5626290,5658890,GGUCUAUUUGGAAGUUGCAUUGUAGAUUUAAGCCCACCAAAUAGGA...,.......(((((((((((((((((...(((((((((....(((((....,3.278190e-07,False,4.784895,1.837604,2.497810,...,11.380905,0.0,6.117607,2.233113,5.444900,0.535760,0.233539,3.514282,2.507485,3.213905
MSTRG.1000.2,chr2L,5626290,5636058,UUCUUGUAGUGCUGUUGAGAUUUUCCGAGUUGCAUAAGCAAUGAGA...,..(((((((((((((((((((....(..(((((....)))))..)(...,9.361186e-09,False,0.260751,1.589011,0.222691,...,0.296748,0.0,0.511957,0.205168,0.000000,0.189093,0.000000,0.291882,0.213929,0.436912
MSTRG.1000.4,chr2L,5626347,5658890,GGUCUAUUUGGAAGUUGCAUUGUAGAUUUAAGCCCACCAAAUAGGA...,(((..(((((((.(((((((((((...(((((((((....(((((....,3.271565e-07,False,1.703695,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.609230,2.605345,7.899505,1.187314,0.000000,2.288152
MSTRG.1000.5,chr2L,5626398,5658890,GGUCUAUUUGGAAGUUGCAUUGUAGAUUUAAGCCCACCAAAUAGGA...,(((((..(((...(((((((((((...(((((((((....(((((....,3.230030e-07,False,5.470272,5.600809,6.753336,...,0.000000,0.0,2.476769,3.637954,0.000000,0.000000,0.000000,2.635837,2.282169,2.674004
MSTRG.10000.1,chr3L,2977026,2979291,CGUCAGCACGACAUGUGAAAAGUGUUGCCUACUUCUAGGGACCUGU...,(((((..(((((((((((...((((((((((....))))..((((....,3.255747e-06,False,2.855986,2.353040,0.737120,...,3.563690,0.0,4.757841,1.319051,0.225422,0.563827,0.000000,1.648786,1.072762,1.549641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MSTRG.9998.1,chr3L,2961522,2964582,GACUAAUUAACCACAGGACAAGUCGAGAAUCACAUCACUCACCCUU...,............(((((((((...((((.((.(((((............,3.976112e-05,False,0.000000,0.000000,0.000000,...,0.943551,0.0,0.000000,0.668902,0.000000,0.000000,0.000000,0.319694,0.000000,0.678013
MSTRG.9998.2,chr3L,2961532,2964582,CCACAGGACAAGUCGAGAAUCACAUCACUCACCCUUCCCAGCCUGC...,.((((((((((...((((.((.(((((...........(((..(((...,4.215752e-05,False,1.183274,1.000688,0.329197,...,1.094784,0.0,1.477451,0.336490,0.534362,0.122293,0.395582,0.676598,0.597496,0.464525
MSTRG.9998.3,chr3L,2961533,2966267,CACAGGACAAGUCGAGAAUCACAUCACUCACCCUUCCCAGCCUGCC...,((((((((((...((((.((.(((((...........(((..((((...,2.644122e-06,False,1.602211,2.832331,0.965503,...,1.882571,0.0,3.821954,1.200196,0.716736,0.450097,0.590028,1.450530,1.324599,1.071220


In [17]:
res.to_excel("flync_vu_gse199164_3d_vs_7d_inference_results.xlsx", index=True)